# CO2 Emissions vs Climated-related Natural Disaster Frequency
<br/>

In [232]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Datasets

### EM-DAT natural disasters dataset (aggregated disaster counts)

**Note**: The specific data exploration, processing and visualization code for the EM-DAT dataset is in the [EM-DAT Data Exploration notebook](https://github.com/NLSanyu/CA682-Data-Visualization-Assignment/blob/main/EM-DAT%20Data%20Exploration.ipynb)

In [233]:
emdat_df = pd.read_csv("data/emdat_disaster_counts_2000_2021.csv", index_col=0)
emdat_df.head()

,Country,ISO,Disaster count,Continent
0,China,CHN,436,Asia
1,Afghanistan,AFG,92,Asia
2,Bangladesh,BGD,105,Asia
3,Bolivia (Plurinational State of),BOL,48,Americas
4,Brazil,BRA,104,Americas


### Our World in Data CO2 emissions dataset

In [234]:
co2_df = pd.read_csv("data/co2.csv")
co2_df.head()

,Entity,Code,Year,Annual CO₂ emissions (per capita)
0,Afghanistan,AFG,1949,0.001922
1,Afghanistan,AFG,1950,0.011266
2,Afghanistan,AFG,1951,0.012098
3,Afghanistan,AFG,1952,0.011946
4,Afghanistan,AFG,1953,0.013685


### Our World in Data land area dataset

In [235]:
la_df = pd.read_csv("data/land-area-km.csv")
la_df.head()

,Entity,Code,Year,Land area (sq. km)
0,Afghanistan,AFG,1961,652860.0
1,Afghanistan,AFG,1962,652860.0
2,Afghanistan,AFG,1963,652860.0
3,Afghanistan,AFG,1964,652860.0
4,Afghanistan,AFG,1965,652860.0


### Our World in Data population dataset

In [236]:
pop_df = pd.read_csv("data/population.csv")
pop_df.head()

,Entity,Code,Year,Population (historical estimates)
0,Afghanistan,AFG,-10000,14737
1,Afghanistan,AFG,-9000,20405
2,Afghanistan,AFG,-8000,28253
3,Afghanistan,AFG,-7000,39120
4,Afghanistan,AFG,-6000,54166


# <br/> Data Processing and Visualization

### Prepare datasets for merging

#### Rename EM-DAT ISO column to match the name of the same column in the CO2 dataset:

In [237]:
emdat_df.rename(columns = {"ISO": "Code"}, inplace=True)
emdat_df.head()

,Country,Code,Disaster count,Continent
0,China,CHN,436,Asia
1,Afghanistan,AFG,92,Asia
2,Bangladesh,BGD,105,Asia
3,Bolivia (Plurinational State of),BOL,48,Americas
4,Brazil,BRA,104,Americas


#### Get only the latest value of country land area from the land area dataset:

In [238]:
la_df["Year"].unique()

array([1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971,
       1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982,
       1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993,
       1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004,
       2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015,
       2016, 2017, 2018, 2019, 2020, 2021])

In [239]:
la_df = la_df[la_df["Year"] == 2021]
la_df["Year"].unique()

array([2021])

In [240]:
la_df.dtypes

Entity                 object
Code                   object
Year                    int64
Land area (sq. km)    float64
dtype: object

In [241]:
la_df["Land area (sq. km)"] = la_df["Land area (sq. km)"].astype("int64")

In [242]:
la_df.head()

,Entity,Code,Year,Land area (sq. km)
60,Afghanistan,AFG,2021,652860
121,Albania,ALB,2021,27400
182,Algeria,DZA,2021,2381741
243,American Samoa,ASM,2021,200
304,Andorra,AND,2021,470


#### Get only the latest value of country population from the population dataset:

In [243]:
pop_df = pop_df[pop_df["Year"] == 2021]
pop_df["Year"].unique()

array([2021])

In [244]:
pop_df.rename(columns={"Population (historical estimates)": "Population"}, inplace=True)

In [245]:
pop_df.dtypes

Entity        object
Code          object
Year           int64
Population     int64
dtype: object

### Data subset choice on the CO2 dataset (to match the subset used for the EM-DAT data in its separate [data exploration notebook](https://github.com/NLSanyu/CA682-Data-Visualization-Assignment/blob/main/EM-DAT%20Data%20Exploration.ipynb))

In [246]:
chosen_years = [
    2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 
    2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021
]

In [247]:
co2_df = co2_df[co2_df["Year"].isin(chosen_years)]
co2_df.head()

,Entity,Code,Year,Annual CO₂ emissions (per capita)
51,Afghanistan,AFG,2000,0.053581
52,Afghanistan,AFG,2001,0.054300
53,Afghanistan,AFG,2002,0.063856
54,Afghanistan,AFG,2003,0.068871
55,Afghanistan,AFG,2004,0.052529


In [248]:
co2_df["Year"].unique()

array([2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021])

### Method 1: Merge the datasets using Pandas

#### First group the emissions data by country then exctract the max emissions value for each country

In [249]:
co2_agg = co2_df.groupby(by=["Entity", "Code"]).agg({"Annual CO₂ emissions (per capita)": "max"})
co2_agg.reset_index(inplace=True)
co2_agg.rename(columns={"Annual CO₂ emissions (per capita)": "Highest annual CO₂ emissions (tonnes per capita)"}, inplace=True)
co2_agg.head()

,Entity,Code,Highest annual CO₂ emissions (tonnes per capita)
0,Afghanistan,AFG,0.404740
1,Albania,ALB,2.080253
2,Algeria,DZA,4.203331
3,Andorra,AND,7.924738
4,Angola,AGO,1.250241


#### Merge the disaster dataset with the CO2 dataset

In [250]:
merged_df_1 = co2_agg.merge(emdat_df, how="inner", on="Code")
merged_df_1.head()

,Entity,Code,Highest annual CO₂ emissions (tonnes per capita),Country,Disaster count,Continent
0,Afghanistan,AFG,0.404740,Afghanistan,92,Asia
1,Albania,ALB,2.080253,Albania,15,Europe
2,Algeria,DZA,4.203331,Algeria,38,Africa
3,Angola,AGO,1.250241,Angola,49,Africa
4,Anguilla,AIA,11.973856,Anguilla,1,Americas


#### Merge the resulting dataset with the land area dataset

In [251]:
merged_df_2 = merged_df_1.merge(pop_df)
merged_df_2["Population"]

0      40099460
1       2854710
2      44177964
3      34503776
4         15779
         ...   
195    97468024
196       11654
197    32981644
198    19473132
199    15993525
Name: Population, Length: 200, dtype: int64

In [252]:
merged_df_2.head()

,Entity,Code,Highest annual CO₂ emissions (tonnes per capita),Country,Disaster count,Continent,Year,Population
0,Afghanistan,AFG,0.404740,Afghanistan,92,Asia,2021,40099460
1,Albania,ALB,2.080253,Albania,15,Europe,2021,2854710
2,Algeria,DZA,4.203331,Algeria,38,Africa,2021,44177964
3,Angola,AGO,1.250241,Angola,49,Africa,2021,34503776
4,Anguilla,AIA,11.973856,Anguilla,1,Americas,2021,15779


#### Bubble chart on merged data

In [253]:
import plotly.express as px

fig = px.scatter(
    merged_df_2, x="Highest annual CO₂ emissions (tonnes per capita)", y="Disaster count", 
    size="Population", color="Continent", hover_name="Country", size_max=50,
)
fig.update_layout(
    title_text="Climate-related disaster counts vs CO2 emissions per capita (2000-2021)",
    font_family="arial, sans-serif"
)
fig.show()

### Method 2: Calculating median on Our World in Data CO2 emissions dataset 

**Note**: This median calculation is done on a subset `(2000-2021)` of the `C02 emissions dataset` that has been filtered using `Excel`. The median is calculated in order to get only `one value` for C02 emissions per country. The resulting dataset with the median is exported into a csv file in order to merge it with the `EM-DAT` dataset using `Excel` and `OpenRefine`

In [254]:
df_3 = pd.read_excel("data/annual-co-emissions-by-region2000.xlsx")
df_3.head()

/opt/homebrew/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:221: UserWarning:

Workbook contains no default style, apply openpyxl's default



,Entity,Code,Year,Annual CO₂ emissions (zero filled)
0,Afghanistan,AFG,2000,1047127.94
1,Afghanistan,AFG,2001,1069098.00
2,Afghanistan,AFG,2002,1340995.00
3,Afghanistan,AFG,2003,1559602.00
4,Afghanistan,AFG,2004,1237247.00


In [255]:
grouped_df = df_3.groupby(by="Entity").agg({"Annual CO₂ emissions (zero filled)": "median"})
grouped_df

,Annual CO₂ emissions (zero filled)
Entity,
Afghanistan,8.716201e+06
Africa,1.233028e+09
Africa (GCP),1.233017e+09
Aland Islands,0.000000e+00
Albania,4.625543e+06
...,...
Western Sahara,0.000000e+00
World,3.392568e+10
Yemen,1.786422e+07


In [256]:
grouped_df.to_csv("data/annual_co2_median.csv")

#### Visualization on the dataset merged using Excel and OpenRefine

In [257]:
df_2 = pd.read_excel("data/CO2_median_disaster_2000_2021.xlsx")
df_2.head()


,Entity,Code,Continent,Disaster count,median CO₂ emissions,land area
0,Afghanistan,AFG,Asia,92.0,8.716201e+06,652860.0
1,Albania,ALB,Europe,15.0,4.625543e+06,27400.0
2,Algeria,DZA,Africa,38.0,1.199962e+08,2381741.0
3,Andorra,AND,Europe,NaN,5.056320e+05,470.0
4,Angola,AGO,Africa,49.0,2.354180e+07,1246700.0


In [258]:
df_2.rename(columns={"median CO₂ emissions": "Median CO₂ emissions"}, inplace=True)

#### Bubble chart on data merged using OpenRefine

In [259]:
import plotly.express as px

fig = px.scatter(df_2, x="Median CO₂ emissions", y="Disaster count", 
                 size="land area", color="Continent", hover_name="Entity")
fig.update_layout(
    title_text="Disaster counts vs Median CO2 emissions",
    font_family="arial, sans-serif"
)
fig.show()